In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch_snippets


In [ ]:
from torch_snippets import *
from torchvision import transforms
from sklearn.model_selection import train_test_split
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2 
import glob

In [ ]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])

In [ ]:
images=sorted(glob.glob('/kaggle/input/sisvse-dataset/miccai2022_sisvse_dataset/images/real/*.jpg'))
train_labels1=sorted(glob.glob('/kaggle/input/sisvse-dataset/miccai2022_sisvse_dataset/semantic_masks/real_train_1/*.png'))
train_labels2=sorted(glob.glob('/kaggle/input/sisvse-dataset/miccai2022_sisvse_dataset/semantic_masks/real_train_1/*.png'))
train_labels3=sorted(glob.glob('/kaggle/input/sisvse-dataset/miccai2022_sisvse_dataset/semantic_masks/real_train_1/*.png'))
train_labels=train_labels2
test_labels=sorted(glob.glob('/kaggle/input/sisvse-dataset/miccai2022_sisvse_dataset/semantic_masks/real_val_2/*.png'))



In [ ]:
def find_image(name,directory):
    name=name.split('/')[-1].split('.')[0]
    for i in directory:
        if name in i:
            return i 
    
find_image(test_labels[10],images)

In [ ]:
class SegData(Dataset):
    def __init__(self,labels):
        self.images=images
        self.labels=labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, ix):
        path=self.labels[ix]
        image_path=find_image(path,self.images)
        image = cv2.imread(image_path)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (512,512))
        mask = read(path,1)
        mask = cv2.resize(mask, (512,512))
        mask=cv2.cvtColor(mask,cv2.COLOR_RGB2GRAY)
        return image, mask
    def choose(self): return self[randint(len(self.labels))]
    def collate_fn(self, batch):
        ims, masks = list(zip(*batch))
        ims = torch.cat([tfms(im.copy()/255.)[None] for im in ims]).float().to(device)
        ce_masks = torch.cat([torch.tensor(mask)[None] for mask in masks]).long().to(device)
        return ims, ce_masks

In [ ]:
train_dataset=SegData(labels=train_labels)
test_dataset=SegData(labels=test_labels)

In [ ]:
train_dataset.__len__()

In [ ]:
image,mask=train_dataset[-100]
print(image.shape)
print(mask.shape)
plt.imshow(image)
plt.figure()
plt.imshow(mask)
print(image.shape)
print(mask)

In [ ]:
trn_dl = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=train_dataset.collate_fn,drop_last=True)
test_dl=DataLoader(test_dataset,batch_size=4,shuffle=True,collate_fn=test_dataset.collate_fn,drop_last=True)

In [ ]:
img,mask=next(iter(trn_dl))
img.shape

In [ ]:
def conv(in_channels, out_channels):
    return nn.Sequential(
        nn.ReflectionPad2d(1),
        nn.Conv2d(in_channels, out_channels, kernel_size=3),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
def up_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

In [ ]:
class tinyUNet(nn.Module):
    def __init__(self, pretrained=True, out_channels=32):
        super().__init__()
        self.block1=conv(3,64)
        self.block2=conv(64,64)
        self.block3=conv(64,128)
        self.block4=conv(128,128)
        self.block5=conv(128,256)
        self.block6=conv(256,256)
        self.block7=conv(256,512)
        self.block8=conv(512,512)
        self.block9=conv(512,1024)
        self.maxpool=nn.MaxPool2d(kernel_size=(2,2),stride=2,padding=0,dilation=1,ceil_mode=False)
        self.up_conv6 = up_conv(1024, 512)
        self.conv6 = conv(512 + 512, 512)
        self.up_conv7 = up_conv(512, 256)
        self.conv7 = conv(256+256,256)
        self.up_conv8 = up_conv(128+128, 128)
        self.conv8 = conv(128 + 128, 128)
        self.up_conv9 = up_conv(128, 64)
        self.conv9 = conv(64 +64, 64)
        self.up_conv10 = up_conv(64, 32)
        self.conv10 = conv(32 + 64, 32)
        self.conv11 = nn.Conv2d(64, out_channels, kernel_size=1)
    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.maxpool(self.block2(block1))
        block3 = self.block3(block2)
        block4 = self.maxpool(self.block4(block3))
        block5 = self.block5(block4)
        block6 = self.maxpool(self.block6(block5))
        block7 = self.block7(block6)
        block8 = self.maxpool(self.block8(block7))
        block9=self.block9(block8)

        x4 = self.up_conv6(block9)
        x4 = torch.cat([x4,block7],dim=1)
        x4=self.conv6(x4)
        x3 = self.up_conv7(x4)
        
        x3 = torch.cat([x3, block5], dim=1)
        x3 = self.conv7(x3)

        x2 = self.up_conv8(x3)
        x2 = torch.cat([x2, block3], dim=1)
        x2 = self.conv8(x2)

        x1 = self.up_conv9(x2)
        x1 = torch.cat([x1, block1], dim=1)
        x1 = self.conv9(x1)

        #x = self.up_conv10(x1)
        #x = torch.cat([x, block1], dim=1)
        #x = self.conv10(x)

        x = self.conv11(x1)

        return x

In [ ]:
model=tinyUNet().to(device)
#model.load_state_dict(torch.load('/kaggle/working/real_model1.pt'))

In [ ]:
%%capture 
!pip install torchsummary

In [ ]:
from torchsummary import summary
summary(model,(3,512,512))

In [ ]:
ce = nn.CrossEntropyLoss()
def iou(preds,targets):
  smooth=1.
  _preds=torch.argmax(torch.nn.Softmax(dim=-1)(preds),1)
  intersection=(_preds==targets).float().sum()
  dice=(intersection)/(_preds.sum()+targets.sum())
  return dice
def UnetLoss(preds, targets):
  loss = ce(preds, targets)
  acc=(torch.argmax(torch.nn.Softmax(dim=-1)(preds),1)==targets).float().mean()
  return loss, acc

In [ ]:
def train_batch(model, data, optimizer, criterion):
    model.train()
    ims, ce_masks = data
    _masks = model(ims)
    optimizer.zero_grad()
    loss, acc = criterion(_masks, ce_masks)
    loss.backward()
    optimizer.step()
    return loss.item(), acc.item()

@torch.no_grad()
def validate_batch(model, data, criterion):
    model.eval()
    ims, masks = data
    _masks = model(ims)
    loss, acc = criterion(_masks,masks)
    return loss.item(), acc.item()

In [ ]:
criterion = UnetLoss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4,weight_decay=1e-6)
n_epochs = 20

In [ ]:

'''
For Training Process,First Train your network with first subset of training dataset with 10 epochs,in the next step use second subset
of dataset with 5 epochs.
'''
log = Report(n_epochs)
for ex in range(n_epochs):
    N = len(trn_dl)
    for bx, data in enumerate(trn_dl):
        loss, acc = train_batch(model, data, optimizer, criterion)
        log.record(ex+(bx+1)/N, trn_loss=loss, trn_acc=acc, end='\r')

    N = len(test_dl)
    for bx, data in enumerate(test_dl):
        loss, acc = validate_batch(model, data, criterion)
        log.record(ex+(bx+1)/N, val_loss=loss, val_acc=acc, end='\r')
        
    log.report_avgs(ex+1)

In [ ]:
img,mask=next(iter(test_dl))
_mask=model(img)
_,_mask=torch.max(_mask,dim=1)
for i in range(4-1):
    subplots([img[i].permute(1,2,0).detach().cpu()[:,:,i],_mask.permute(1,2,0).detach().cpu()[:,:,i],mask.permute(1,2,0).detach().cpu()[:,:,i]])

In [ ]:
PATH = "real_model1_onnx_compat.pt"
torch.save(model.state_dict(), PATH)